# Part 1

Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import json 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans 
from bs4 import BeautifulSoup 
from geopy.geocoders import Nominatim 
import folium

Web scraping

In [ ]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [ ]:
soup = BeautifulSoup(data, 'html.parser')

Data formating

In [ ]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


Creation of the dataframe

In [ ]:
toronto_df = pd.DataFrame(table_contents)
toronto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Data Cleaning

In [ ]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [ ]:
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [ ]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [ ]:
toronto_df_grouped.shape

(103, 3)

Question 1

In [ ]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


# Part 2

In [ ]:
!pip install geocoder

     |████████████████████████████████| 102kB 5.7MB/s 


In [ ]:
import geocoder

In [ ]:
toronto_df_grouped.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,15,103
top,M9P,North York,Church and Wellesley
freq,1,24,1


Function that gets the coordinates from a postal code

In [ ]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.709020000000066, -79.36348999999996]

Extracting all postal codes coordinates

In [ ]:
postal_codes = toronto_df_grouped['PostalCode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

Adding these coordinates to the dataframe

In [ ]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
toronto_df_grouped['Latitude'] = df_coords['Latitude']
toronto_df_grouped['Longitude'] = df_coords['Longitude']

In [ ]:
toronto_df_grouped[toronto_df_grouped.PostalCode == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493


Question 2

In [ ]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
1,M2H,North York,Hillcrest Village,43.80225,-79.35558
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
3,M1J,Scarborough,Scarborough Village,43.74446,-79.23117
4,M4G,East York,Leaside,43.70902,-79.36349
5,M4M,East Toronto,Studio District,43.66213,-79.33497
6,M1R,Scarborough,"Wexford, Maryvale",43.75075,-79.30054
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.74453,-79.58624
8,M9L,North York,Humber Summit,43.75948,-79.55707
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.64082,-79.39818


In [ ]:
toronto_df_grouped.shape

(103, 5)

#Part 3

Extracting Coordinates from Toronto, Ontario Canada

In [ ]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


Creating a map of Toronto, with each bluemark representing a borough with its neighborhoods

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(toronto_df_grouped['Latitude'], toronto_df_grouped['Longitude'], toronto_df_grouped['Borough'], toronto_df_grouped['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

In [ ]:
toronto_data = toronto_df_grouped[toronto_df_grouped['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.67709,-79.29547
1,M4J,East YorkEast Toronto,The Danforth East,43.68811,-79.33418
2,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35512
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.66797,-79.31467
4,M4M,East Toronto,Studio District,43.66213,-79.33497


Creating a map of Toronto, with each bluemark representing a neighborhood

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_toronto)  
map_toronto

Foursqaure API

In [1]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Function to retrive the nearby venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Extracting the venues for each neighborhood

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

In [ ]:
print(len(toronto_data['Neighborhood']))

39


1699 Venues in 39 neighborhoods

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

(1699, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.67709,-79.29547,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.67709,-79.29547,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.67709,-79.29547,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.67709,-79.29547,Domino's Pizza,43.679058,-79.297382,Pizza Place
4,The Beaches,43.67709,-79.29547,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


Number of venues per neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,63,63,63,63,63,63
"Brockton, Parkdale Village, Exhibition Place",85,85,85,85,85,85
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",81,81,81,81,81,81
Central Bay Street,63,63,63,63,63,63
Christie,11,11,11,11,11,11
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,29,29,29,29,29,29
Davisville North,7,7,7,7,7,7


In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 223 uniques categories.


Analysing venues per categories

In [ ]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
toronto_onehot.shape

(1699, 223)

Analysing the distribition of venues per neighborhood and category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.015873,0.000000,0.000000,0.015873,0.00,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.047619,0.000000,0.000000,0.00,0.00,0.000000,0.031746,0.000000,0.015873,0.00,0.015873,0.000000,0.00,0.000000,0.000000,0.00,0.031746,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.015873,0.000000,...,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.00,0.015873,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.0000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.023529,0.011765,0.000000,0.000000,0.00,0.000000,0.000000,0.023529,0.000000,0.000000,0.011765,0.000000,0.000000,0.023529,0.000000,0.058824,0.00,0.00,0.000000,0.011765,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.023529,0.011765,0.000000,0.000000,0.023529,0.000000,0.000,0.000000,0.058824,0.000000,...,0.000000,0.011765,0.000000,0.000000,0.000000,0.011765,0.000000,0.000000,0.011765,0.00,0.011765,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,0.000000,0.023529,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,0.000000,0.000000,0.011765,0.000000,0.000000,0.000000,0.0000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.012346,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.024691,0.012346,0.037037,0.00,0.00,0.000000,0.012346,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.012346,0.000000,0.012346,0.000000,0.000000,0.000,0.000000,0.061728,0.000000,...,0.000000,0.012346,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.00,0.012346,0.024691,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.00,0.015873,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.031746,0.000000,0.00,0.000000,0.000000,0.031746,0.000000,0.000000,0.000000,0.000,0.000000,0.015873,0.000000,...,0.000000,0.000000,0.015873,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.015873,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.00,0.000000,0.000000,0.015873,0.000000,0.000000,0.015873,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.015873,0.015873,0.0000,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000

In [ ]:
toronto_grouped.shape

(39, 223)

Get the top 5 categories for each neighborhood

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.06
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.05
3              Bakery  0.05
4         Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0          Bar  0.06
1         Café  0.06
2   Restaurant  0.05
3  Coffee Shop  0.05
4    Gift Shop  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.06
2                Café  0.06
3                Park  0.04
4                 Bar  0.04


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.11
1  Clothing Store  0.10
2  Sandwich Place  0.05
3       Bookstore  0.03
4           Plaza  0.03


----Christie----
                venue  freq
0                Café  0.27
1       Grocery Store  0.18
2         Coffee Shop  0.09
3  Italian Restaurant  0.09
4          Bab

Function to get the most frequent venues in a neighborhood

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Bakery,Beer Bar,Restaurant,Farmers Market,Breakfast Spot,Pharmacy,Cheese Shop
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Restaurant,Coffee Shop,Sandwich Place,Gift Shop,Italian Restaurant,Breakfast Spot,Nightclub,Lounge
2,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Café,Coffee Shop,French Restaurant,Park,Bar,Intersection,Speakeasy,Lounge,Restaurant
3,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Bookstore,Cosmetics Shop,Bubble Tea Shop,Plaza,Pizza Place,Shopping Mall
4,Christie,Café,Grocery Store,Candy Store,Coffee Shop,Baby Store,Athletics & Sports,Playground,Italian Restaurant,Ethiopian Restaurant,Donut Shop


Defining the K-Means Clustering

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Creating a dataframe with the cluster label and the most common venues

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged =toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.67709,-79.29547,1,Health Food Store,Asian Restaurant,Pizza Place,Pub,Trail,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Escape Room
1,M4J,East YorkEast Toronto,The Danforth East,43.68811,-79.33418,0,Park,Intersection,Wings Joint,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
2,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35512,1,Discount Store,Bus Line,Intersection,Ice Cream Shop,Coffee Shop,Business Service,Grocery Store,Park,Event Space,Falafel Restaurant
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.66797,-79.31467,1,Pizza Place,Park,Fast Food Restaurant,Restaurant,Italian Restaurant,Sushi Restaurant,Food & Drink Shop,Board Shop,Ice Cream Shop,Fish & Chips Shop
4,M4M,East Toronto,Studio District,43.66213,-79.33497,1,Brewery,Italian Restaurant,Diner,Pizza Place,Sushi Restaurant,Coffee Shop,Bar,Arts & Crafts Store,Bakery,American Restaurant


Graphical representation of the clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East YorkEast Toronto,0,Park,Intersection,Wings Joint,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
35,West Toronto,0,Park,Wings Joint,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Health Food Store,Asian Restaurant,Pizza Place,Pub,Trail,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Escape Room
2,East Toronto,1,Discount Store,Bus Line,Intersection,Ice Cream Shop,Coffee Shop,Business Service,Grocery Store,Park,Event Space,Falafel Restaurant
3,East Toronto,1,Pizza Place,Park,Fast Food Restaurant,Restaurant,Italian Restaurant,Sushi Restaurant,Food & Drink Shop,Board Shop,Ice Cream Shop,Fish & Chips Shop
4,East Toronto,1,Brewery,Italian Restaurant,Diner,Pizza Place,Sushi Restaurant,Coffee Shop,Bar,Arts & Crafts Store,Bakery,American Restaurant
6,Central Toronto,1,Food & Drink Shop,Gym,Park,Breakfast Spot,Playground,Hotel,Department Store,Wings Joint,Dumpling Restaurant,Eastern European Restaurant
8,Central Toronto,1,Dessert Shop,Italian Restaurant,Sandwich Place,Café,Coffee Shop,Pizza Place,Thai Restaurant,Seafood Restaurant,Farmers Market,Diner
9,Central Toronto,1,Gym,Convenience Store,Tennis Court,Wings Joint,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
10,Central Toronto,1,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Wings Joint,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Event Space
12,Downtown Toronto,1,Park,Coffee Shop,Restaurant,Italian Restaurant,Café,Pub,Market,Pizza Place,Bakery,Sri Lankan Restaurant
13,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Café,Hotel,Fast Food Restaurant,Dance Studio,Gay Bar,Grocery Store


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,2,Gym Pool,Playground,Park,Boutique,Grocery Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
11,Downtown Toronto,2,Park,Playground,Bike Trail,Wings Joint,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,3,Fast Food Restaurant,Home Service,Wings Joint,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,4,Bus Line,Lake,Swim School,Wings Joint,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
